# Basic dataverk usage
This example shows simple dataverk usage for ETL:
- Retrieving data from a web-endpoint
- Transforming the data
- Packaging the data resources and metadata
- Publishing the resulting package  

## Import libraries

In [ ]:
import plotly.graph_objs as go
import plotly.io as plio

from dataverk import Client, Datapackage

## Instanciate context object

In [ ]:
dv = Client()

## Read data

In [ ]:
df = dv.read_json_stat('https://data.ssb.no/api/v0/no/table/09665')

## View dataframe
View the first five rows of dataset

In [ ]:
df.head(5)

## Transform data
Transforming the data into separate dataframes

In [ ]:
df_e = df[(df['kjønn']!='Begge kjønn') & (df['statistikkvariabel']=='Sykefraværsprosent, ikke-sesongjustert')]
df_m = df[(df['kjønn']=='Menn') & (df['type sykefravær']=='Egenmeldt og legemeldt') & (df['statistikkvariabel']=='Sykefraværsprosent, ikke-sesongjustert')]
df_k = df[(df['kjønn']=='Kvinner') & (df['type sykefravær']=='Egenmeldt og legemeldt') & (df['statistikkvariabel']=='Sykefraværsprosent, ikke-sesongjustert')]

## Create data views
Create a scatter plot

In [ ]:
data = [ go.Scatter(
            x=df_m['kvartal'], 
            y=df_m['value'],
            name='Menn'
            ),
        go.Scatter(
            x=df_k['kvartal'], 
            y=df_k['value'],
            name='Kvinner'
            )
       ]

layout = go.Layout(title='Sykefravær i norge')
fig_1 = go.Figure(data,layout)

go.FigureWidget(data,layout)

Create a barchart

In [ ]:
data = [ go.Bar(
     x=df_m['kvartal'], 
     y=df_m['value'],
     name='Menn'
),
go.Bar(
    x=df_k['kvartal'], 
    y=df_k['value'],
    name='Kvinner'
)
]

layout = go.Layout(
    barmode='stack'
)

layout = go.Layout(title='Sykefravær')
fig_2 = go.Figure(data,layout)

go.FigureWidget(data,layout)

## Create datapackage
Instanciate the datapackage object with metadata

In [ ]:
readme = """
# SSB Sykefravær
Sykefravær for lønnstakere, etter kjønn og type sykefravær. Sesongjustert og ikke-sesongjustert (prosent) (avslutta serie) 2000K2 - 2018K4

kilde: ['SSB tabell 09665']('https://www.ssb.no/statbank/table/09665')
"""

metadata = {
    "title": "SSB Sykefravær",
    "readme": readme,
    "licence": "MIT",
    "auth": "NAV",
    "provenance": "SSB",
    "store": "gs",
    "bucket": "nav-opendata",
    "description": "SSB Sykefravær",
    "keywords": ["SSB", "Sykefravær", ""],
    "publisher": "NAV"
}

dp = Datapackage(metadata)

## Add resources

In [ ]:
dp.add_resource(df_e, "Egenmeldt og legemeldt")
dp.add_resource(df_m, "Sykefravær menn egenmeldt og legemeldt")
dp.add_resource(df_k, "Sykefravær kvinner egenmeldt og legemeldt")

## Add views
Add scatter plot view to datapackage

In [ ]:
desc = '''
### Sykefravær.

Egenmeldt og legemeldt. Sykefraværsprosent, ikke-sesongjustert
'''

dp.add_view(
    spec_type='plotly',
    name='Sykefravær',
    title='Sykefravær',
    description =desc,
    spec= plio.to_json(fig_1),
    resources=''
)

Add barchart view to datapackage

In [ ]:
desc = '''
### Sykefravær.

Egenmeldt og legemeldt. Sykefraværsprosent, ikke-sesongjustert
'''

dp.add_view(
    spec_type='plotly',
    name='Sykefravær',
    title='Sykefravær',
    description =desc,
    spec= plio.to_json(fig_2),
    resources=''
)

## Publish data to bucket storage and metadata to es index

In [ ]:
# dv.publish(dp)